# INST326 — Week 9 Exercises: Abstract Classes & Interfaces (Library Management)

**Focus (Week 9 only):** Abstract Base Classes (ABCs) with `abc.ABC` and `@abstractmethod`, abstract properties, virtual subclass registration, and light “interface-like” design via ABCs and (optional) `typing.Protocol` **without** advanced generics.

**Out of scope (Week 10+):** multiple inheritance/mixins, advanced design patterns, dependency injection, metaclasses beyond `ABCMeta`, decorators beyond basics, complex type-system features (ParamSpec, TypeVar variance), context managers beyond prior weeks.


### Starter Scaffold (Week-9-safe)

Below is a minimal domain model from prior weeks, slightly adapted for Week 9. We keep inheritance simple and introduce **abstract classes** to define common contracts.


In [ ]:
from __future__ import annotations
from dataclasses import dataclass
from datetime import datetime, timedelta
from typing import Optional, List, Dict, Protocol
from abc import ABC, abstractmethod

# --- Exceptions (kept simple) ---
class LibraryError(Exception): ...
class DuplicateBookError(LibraryError): ...
class OverdueLoanError(LibraryError): ...
class NonBorrowableError(LibraryError): ...

# --- Abstract base for library items ---
class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""

    # A partial template method that depends on an abstract method.
    def due_date_from_today(self) -> datetime:
        return datetime.now() + timedelta(days=self.loan_period_days())

    # default stock policy common to most items
    def can_checkout(self) -> bool:
        return self.copies > 0

# --- Concrete items (will be extended in exercises) ---
class PrintedBook(LibraryItem):
    def loan_period_days(self) -> int:
        return 21
    def describe(self) -> str:
        return f"PrintedBook<{self.isbn}>: {self.title} (copies={self.copies})"
    @property
    def is_digital(self) -> bool:
        return False

class EBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 0, file_size_mb: float = 0.0) -> None:
        super().__init__(isbn, title, copies)
        self.file_size_mb = float(file_size_mb)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb:.1f} MB)"
    @property
    def is_digital(self) -> bool:
        return True
    # Different stock rule (licenses can be zero but not negative)
    def can_checkout(self) -> bool:
        return self.copies >= 0

class AudioBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 1, duration_min: int = 0) -> None:
        super().__init__(isbn, title, copies)
        self.duration_min = int(duration_min)
    def loan_period_days(self) -> int:
        return 14
    def describe(self) -> str:
        return f"AudioBook<{self.isbn}>: {self.title} ({self.duration_min} min)"
    @property
    def is_digital(self) -> bool:
        return True

# --- Optional Protocol example (kept simple & non-generic) ---
class Downloadable(Protocol):
    def download_link(self) -> str: ...

# --- Core containers ---
@dataclass
class Member:
    member_id: str
    email: str
    def max_concurrent_loans(self) -> int:
        return 5

@dataclass
class Loan:
    isbn: str
    member_id: str
    due_date: datetime
    returned: bool = False
    def mark_returned(self) -> None:
        self.returned = True

class Catalog:
    def __init__(self):
        self._items: Dict[str, LibraryItem] = {}
    def add_item(self, item: LibraryItem) -> None:
        if item.isbn in self._items:
            raise DuplicateBookError(f"ISBN already exists: {item.isbn}")
        if item.copies < 0:
            raise ValueError("copies must be non-negative")
        self._items[item.isbn] = item
    def get_item(self, isbn: str) -> Optional[LibraryItem]:
        return self._items.get(isbn)

class LoanDesk:
    def __init__(self, catalog: Catalog):
        self.catalog = catalog
        self.loans: List[Loan] = []
    def active_loans_for(self, member: Member) -> List[Loan]:
        return [L for L in self.loans if (L.member_id == member.member_id and not L.returned)]
    def checkout(self, member: Member, item: LibraryItem) -> Loan:
        if len(self.active_loans_for(member)) >= member.max_concurrent_loans():
            raise LibraryError("concurrent loan limit reached")
        if not item.can_checkout():
            raise NonBorrowableError("cannot checkout under current stock policy")
        item.copies -= 1
        loan = Loan(isbn=item.isbn, member_id=member.member_id, due_date=item.due_date_from_today())
        self.loans.append(loan)
        return loan
    def checkin(self, loan: Loan) -> None:
        if not loan.returned:
            it = self.catalog.get_item(loan.isbn)
            if it:
                it.copies += 1
            loan.mark_returned()


## 1) Make `LibraryItem` truly abstract

Prove that `LibraryItem` cannot be instantiated directly. Write a quick try/except demonstrating that creating `LibraryItem('x','y')` raises a `TypeError` because of abstract methods.

In [ ]:
# Your code here
# Demonstrate TypeError on instantiation of abstract class
try:
    item = LibraryItem('978-0-123456-78-9', 'Some Title')
    print("ERROR: LibraryItem was instantiated (this should not happen!)")
except TypeError as e:
    print("✓ Correctly prevented instantiation of abstract class")
    print(f"  TypeError message: {e}")

## 2) Abstract property practice

Add an **abstract property** `media_type` to `LibraryItem` and implement it in all concrete subclasses with strings like `'print'`, `'ebook'`, `'audiobook'`.

In [ ]:
# Your code here
# Add @property @abstractmethod def media_type(self) -> str: ...
class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""

    @property
    @abstractmethod
    def media_type(self) -> str:
        """Return the media type as a string."""

    # A partial template method that depends on an abstract method.
    def due_date_from_today(self) -> datetime:
        return datetime.now() + timedelta(days=self.loan_period_days())

    # default stock policy common to most items
    def can_checkout(self) -> bool:
        return self.copies > 0


# Implement in PrintedBook:
class PrintedBook(LibraryItem):
    def loan_period_days(self) -> int:
        return 21
    
    def describe(self) -> str:
        return f"PrintedBook<{self.isbn}>: {self.title} (copies={self.copies})"
    
    @property
    def is_digital(self) -> bool:
        return False
    
    @property
    def media_type(self) -> str:
        return 'print'


# Implement in EBook:
class EBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 0, file_size_mb: float = 0.0) -> None:
        super().__init__(isbn, title, copies)
        self.file_size_mb = float(file_size_mb)
    
    def loan_period_days(self) -> int:
        return 14
    
    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb:.1f} MB)"

## 3) Abstract classmethod

Add an abstract `@classmethod def kind(cls) -> str` to `LibraryItem` returning a short identifier (e.g., `'book'`). Implement it for the concrete subclasses.

In [ ]:
# Your code here
# classmethod kind() -> str on LibraryItem and overrides

class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""

    @property
    @abstractmethod
    def media_type(self) -> str:
        """Return the media type as a string."""

    @classmethod
    @abstractmethod
    def kind(cls) -> str:
        """Return a short identifier for this kind of library item."""

    # A partial template method that depends on an abstract method.
    def due_date_from_today(self) -> datetime:
        return datetime.now() + timedelta(days=self.loan_period_days())

    # default stock policy common to most items
    def can_checkout(self) -> bool:
        return self.copies > 0


# Implement in PrintedBook:
class PrintedBook(LibraryItem):
    def loan_period_days(self) -> int:
        return 21
    
    def describe(self) -> str:
        return f"PrintedBook<{self.isbn}>: {self.title} (copies={self.copies})"
    
    @property
    def is_digital(self) -> bool:
        return False
    
    @property
    def media_type(self) -> str:
        return 'print'
    
    @classmethod
    def kind(cls) -> str:
        return 'book'


# Implement in EBook:
class EBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 0, file_size_mb: float = 0.0) -> None:
        super().__init__(isbn, title, copies)
        self.file_size_mb = float(file_size_mb)
    
    def loan_period_days(self) -> int:
        return 14
    
    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb:.1f} MB)"
    
    @property
    def is_digital(self) -> bool:
        return True
    
    def can_checkout(self) -> bool:
        return self.copies >= 0
    
    @property
    def media_type(self) -> str:
        return 'ebook'
    
    @classmethod
    def kind(cls) -> str:
        return 'ebook'


class AudioBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 1, duration_min: int = 0) -> None:
        super().__init__(isbn, title, copies)
        self.duration_min = int(duration_min)
    
    def loan_period_days(self) -> int:
        return 14
    
    def describe(self) -> str:
        return f"AudioBook<{self.isbn}>: {self.title} ({self.duration_min} min)"
    
    @property
    def is_digital(self) -> bool:
        return True
    
    @property
    def media_type(self) -> str:
        return 'audiobook'
    
    @classmethod
    def kind(cls) -> str:
        return 'audiobook'


       

## 4) Template method using abstract hook

Create a template method `receipt_line(self) -> str` on `LibraryItem` that uses `self.describe()` and `self.loan_period_days()`. Show that each subclass inherits the same template but outputs different text due to overrides.

In [ ]:
# Your code here
# Implement receipt_line in LibraryItem using abstract hooks

class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""

    @property
    @abstractmethod
    def media_type(self) -> str:
        """Return the media type as a string."""

    @classmethod
    @abstractmethod
    def kind(cls) -> str:
        """Return a short identifier for this kind of library item."""

    
    def receipt_line(self) -> str:
        return f"{self.describe()} | Loan period: {self.loan_period_days()} days"


    def due_date_from_today(self) -> datetime:
        return datetime.now() + timedelta(days=self.loan_period_days())

    def can_checkout(self) -> bool:
        return self.copies > 0


class PrintedBook(LibraryItem):
    def loan_period_days(self) -> int:
        return 21
    
    def describe(self) -> str:
        return f"PrintedBook<{self.isbn}>: {self.title} (copies={self.copies})"
    
    @property
    def is_digital(self) -> bool:
        return False
    
    @property
    def media_type(self) -> str:
        return 'print'
    
    @classmethod
    def kind(cls) -> str:
        return 'book'


class EBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 0, file_size_mb: float = 0.0) -> None:
        super().__init__(isbn, title, copies)
        self.file_size_mb = float(file_size_mb)
    
    def loan_period_days(self) -> int:
        return 14
    
    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb:.1f} MB)"
    
    @property
    def is_digital(self) -> bool:
        return True
    
    def can_checkout(self) -> bool:
        return self.copies >= 0
    
    @property
    def media_type(self) -> str:
        return 'ebook'
    
    @classmethod
    def kind(cls) -> str:
        return 'ebook'


class AudioBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 1, duration_min: int = 0) -> None:
        super().__init__(isbn, title, copies)
        self.duration_min = int(duration_min)
    
    def loan_period_days(self) -> int:
        return 14
    
    def describe(self) -> str:
        return f"AudioBook<{self.isbn}>: {self.title} ({self.duration_min} min)"
    
    @property
    def is_digital(self) -> bool:
        return True
    
    @property
    def media_type(self) -> str:
        return 'audiobook'
    
    @classmethod
    def kind(cls) -> str:
        return 'audiobook'


## 5) Virtual subclass registration

Create a new class `PDFPamphlet` **without** inheriting from `LibraryItem`, but `register` it as a virtual subclass using `LibraryItem.register(PDFPamphlet)`. Implement the required interface manually. Show that `isinstance(pdf, LibraryItem)` returns `True` after registration.

In [ ]:
# Your code here
# Define PDFPamphlet, register as virtual subclass, demonstrate isinstance

# Define PDFPamphlet WITHOUT inheriting from LibraryItem
class PDFPamphlet:

    
    def __init__(self, isbn: str, title: str, copies: int = 1, pages: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies
        self.pages = pages
    
    def loan_period_days(self) -> int:
        return 7
    
    def describe(self) -> str:
        return f"PDFPamphlet<{self.isbn}>: {self.title} ({self.pages} pages)"
    
    @property
    def is_digital(self) -> bool:
        return True
    
    @property
    def media_type(self) -> str:
        return 'pdf_pamphlet'
    
    @classmethod
    def kind(cls) -> str:
        return 'pamphlet'
    
    def can_checkout(self) -> bool:
        return self.copies > 0
    
    def due_date_from_today(self) -> datetime:
        return datetime.now() + timedelta(days=self.loan_period_days())



LibraryItem.register(PDFPamphlet)



print("=== Virtual Subclass Registration Demo ===\n")


pdf = PDFPamphlet('978-9-999999-99-9', 'Quick Python Reference', copies=10, pages=8)


print(f"isinstance(pdf, PDFPamphlet): {isinstance(pdf, PDFPamphlet)}")
print(f"isinstance(pdf, LibraryItem): {isinstance(pdf, LibraryItem)}")
print()


print(f"PDFPamphlet.__bases__: {PDFPamphlet.__bases__}")
print(f"Is PDFPamphlet a direct subclass? {PDFPamphlet in LibraryItem.__subclasses__()}")
print(f"Is PDFPamphlet registered? {issubclass(PDFPamphlet, LibraryItem)}")
print()


print(f"Description: {pdf.describe()}")
print(f"Loan period: {pdf.loan_period_days()} days")
print(f"Media type: {pdf.media_type}")
print(f"Kind: {pdf.kind()}")
print(f"Is digital: {pdf.is_digital}")
print(f"Can checkout: {pdf.can_checkout()}")
print()


def process_library_item(item: LibraryItem) -> None:
    """Function that accepts any LibraryItem (including virtual subclasses)."""
    print(f"Processing: {item.describe()}")
    print(f"  Loan period: {item.loan_period_days()} days")
    print(f"  Due date: {item.due_date_from_today().strftime('%Y-%m-%d')}")

print("=== Polymorphic Usage ===")
book = PrintedBook('123', 'Real Book', copies=5)
process_library_item(book)
print()
process_library_item(pdf)
print()

print("=== Key Insight ===")
print("PDFPamphlet does NOT inherit from LibraryItem (no 'is-a' relationship),")
print("but it's registered as a virtual subclass, so isinstance() returns True.")
print("This is called 'duck typing with registration' - if it implements the")
print("interface and is registered, Python treats it as a subclass.")

## 6) Abstract property for availability

Add an abstract property `borrowable: bool` to `LibraryItem`. For `PrintedBook` and `AudioBook`, return `True`. For `EBook`, return `True` if `copies >= 0`. Demonstrate a check before `LoanDesk.checkout` that raises `NonBorrowableError` if `borrowable` is False.

In [ ]:
# Your code here
# Add property and demonstrate guarding behavior

class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""

    @property
    @abstractmethod
    def media_type(self) -> str:
        """Return the media type as a string."""

    @property
    @abstractmethod
    def borrowable(self) -> bool:
        """Whether the item can be borrowed."""

    @classmethod
    @abstractmethod
    def kind(cls) -> str:
        """Return a short identifier for this kind of library item."""

    # Template method - uses abstract hooks
    def receipt_line(self) -> str:
        return f"{self.describe()} | Loan period: {self.loan_period_days()} days"

    def due_date_from_today(self) -> datetime:
        return datetime.now() + timedelta(days=self.loan_period_days())

   
    def can_checkout(self) -> bool:
        return self.copies > 0


class PrintedBook(LibraryItem):
    def loan_period_days(self) -> int:
        return 21
    
    def describe(self) -> str:
        return f"PrintedBook<{self.isbn}>: {self.title} (copies={self.copies})"
    
    @property
    def is_digital(self) -> bool:
        return False
    
    @property
    def media_type(self) -> str:
        return 'print'
    
    @property
    def borrowable(self) -> bool:
        """Printed books are always borrowable."""
        return True
    
    @classmethod
    def kind(cls) -> str:
        return 'book'


class EBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 0, file_size_mb: float = 0.0) -> None:
        super().__init__(isbn, title, copies)
        self.file_size_mb = float(file_size_mb)
    
    def loan_period_days(self) -> int:
        return 14
    
    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb:.1f} MB)"
    
    @property
    def is_digital(self) -> bool:
        return True
    
    def can_checkout(self) -> bool:
        return self.copies >= 0
    
    @property
    def media_type(self) -> str:
        return 'ebook'
    
    @property
    def borrowable(self) -> bool:
        return self.copies >= 0
    
    @classmethod
    def kind(cls) -> str:
        return 'ebook'


class AudioBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 1, duration_min: int = 0) -> None:
        super().__init__(isbn, title, copies)
        self.duration_min = int(duration_min)
    
    def loan_period_days(self) -> int:
        return 14
    
    def describe(self) -> str:
        return f"AudioBook<{self.isbn}>: {self.title} ({self.duration_min} min)"
    
    @property
    def is_digital(self) -> bool:
        return True
    
    @property
    def media_type(self) -> str:
        return 'audiobook'
    
    @property
    def borrowable(self) -> bool:
        """AudioBooks are always borrowable."""
        return True
    
    @classmethod
    def kind(cls) -> str:
        return 'audiobook'


class LoanDesk:
    def __init__(self, catalog: Catalog):
        self.catalog = catalog
        self.loans: List[Loan] = []
    
    def active_loans_for(self, member: Member) -> List[Loan]:
        return [L for L in self.loans if (L.member_id == member.member_id and not L.returned)]
    
    def checkout(self, member: Member, item: LibraryItem) -> Loan:
        if not item.borrowable:
            raise NonBorrowableError(f"Item {item.isbn} is not borrowable")
        
        if len(self.active_loans_for(member)) >= member.max_concurrent_loans():
            raise LibraryError("concurrent loan limit reached")
        
        if not item.can_checkout():
            raise NonBorrowableError("cannot checkout under current stock policy")
        
        item.copies -= 1
        loan = Loan(isbn=item.isbn, member_id=member.member_id, due_date=item.due_date_from_today())
        self.loans.append(loan)
        return loan
    
    def checkin(self, loan: Loan) -> None:
        if not loan.returned:
            it = self.catalog.get_item(loan.isbn)
            if it:
                it.copies += 1
            loan.mark_returned()


print("=== Demonstrating borrowable Property ===\n")

catalog = Catalog()
desk = LoanDesk(catalog)
member = Member(member_id="M001", email="user@example.com")


## 7) EBook implements Downloadable Protocol

Implement `download_link(self) -> str` on `EBook` to satisfy `Downloadable`. Write a function `offer_download(x)` that accepts a `Downloadable` and returns its link. Show duck-typed use with an `EBook` instance.

In [ ]:
# Your code here
# EBook.download_link and offer_download(downloadable)



class EBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 0, file_size_mb: float = 0.0) -> None:
        super().__init__(isbn, title, copies)
        self.file_size_mb = float(file_size_mb)
    
    def loan_period_days(self) -> int:
        return 14
    
    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb:.1f} MB)"
    
    @property
    def is_digital(self) -> bool:
        return True
    
    def can_checkout(self) -> bool:
        return self.copies >= 0
    
    @property
    def media_type(self) -> str:
        return 'ebook'
    
    @property
    def borrowable(self) -> bool:
        return self.copies >= 0
    
    @classmethod
    def kind(cls) -> str:
        return 'ebook'
    
    def download_link(self) -> str:
        return f"https://library.example.com/downloads/ebook/{self.isbn}.epub"



def offer_download(x: Downloadable) -> str:
    """Accept any object that satisfies the Downloadable protocol."""
    link = x.download_link()
    return f"Download available at: {link}"


print("=== Demonstrating Downloadable Protocol ===\n")


ebook = EBook('978-0-123456-78-9', 'Python for Beginners', copies=10, file_size_mb=5.2)

print(f"EBook: {ebook.describe()}")
print(f"Is digital: {ebook.is_digital}")
print()


print("Calling offer_download() with EBook instance:")
result = offer_download(ebook)
print(result)
print()


print("Direct download_link() call:")
print(f"Link: {ebook.download_link()}")
print()


print("=== Protocol Checking ===")
print(f"Has download_link method: {hasattr(ebook, 'download_link')}")
print(f"Is callable: {callable(getattr(ebook, 'download_link', None))}")
print()


class PDFDocument:
    """A standalone class that also satisfies Downloadable protocol."""
    def __init__(self, doc_id: str, title: str):
        self.doc_id = doc_id
        self.title = title
    
    def download_link(self) -> str:
        return f"https://library.example.com/downloads/pdf/{self.doc_id}.pdf"



print("=== Duck Typing in Action ===")
pdf = PDFDocument('DOC123', 'Library Catalog 2025')
print(f"PDFDocument: {pdf.title}")
print(offer_download(pdf))
print()


class Magazine:
    def __init__(self, title: str):
        self.title = title



print("=== Type Safety ===")
magazine = Magazine('Tech Weekly')
print(f"Magazine: {magazine.title}")
try:
    result = offer_download(magazine)
    print(result)
except AttributeError as e:
    print(f"✗ Error: {e}")
    print("  Magazine doesn't satisfy Downloadable protocol (missing download_link)")
print()

print("=== Summary ===")
print("The Downloadable Protocol allows duck typing:")
print("- EBook implements download_link() and satisfies the protocol")
print("- offer_download() accepts ANY object with download_link()")
print("- No inheritance required - just implement the method!")
print("- Type checkers like mypy can verify protocol compliance statically")
print("- At runtime, AttributeError is raised if protocol isn't satisfied")


## 8) Protocol vs ABC (short reflection)

In a short markdown cell, explain the difference between using an ABC and a Protocol for “interfaces” in Python, and when you might pick one over the other in this project.

In [ ]:
# (Write your reflection in this markdown cell.)
"""Reflection:
(ABCs) and Protocols serve different purposes when defining interfaces in Python. 
ABCs require explicit inheritance or registration and enforce implementation of abstract methods at instantiation time, making them ideal for core domain entities like LibraryItem 
Protocols use structural subtyping and don't require inheritance.
a class satisfies a Protocol simply by implementing the required methods—making them perfect for optional capabilities like Downloadable.
""" 

## 9) `LoanDesk` typed for abstraction

Refactor type hints in `LoanDesk` to accept `LibraryItem` rather than concrete classes everywhere. Explain (markdown) why depending on the abstract type improves flexibility.

In [ ]:
# Your code here
# (Minor changes may already reflect this in the scaffold.)

class LoanDesk:
    def __init__(self, catalog: Catalog):
        self.catalog = catalog
        self.loans: List[Loan] = []
    
    def active_loans_for(self, member: Member) -> List[Loan]:
        return [L for L in self.loans if (L.member_id == member.member_id and not L.returned)]
    
   
    def checkout(self, member: Member, item: LibraryItem) -> Loan:
        # Check if item is borrowable FIRST
        if not item.borrowable:
            raise NonBorrowableError(f"Item {item.isbn} is not borrowable")
        
        if len(self.active_loans_for(member)) >= member.max_concurrent_loans():
            raise LibraryError("concurrent loan limit reached")
        
        if not item.can_checkout():
            raise NonBorrowableError("cannot checkout under current stock policy")
        
        item.copies -= 1
        loan = Loan(isbn=item.isbn, member_id=member.member_id, due_date=item.due_date_from_today())
        self.loans.append(loan)
        return loan
    
    def checkin(self, loan: Loan) -> None:
        if not loan.returned:
            it: Optional[LibraryItem] = self.catalog.get_item(loan.isbn)
            if it:
                it.copies += 1
            loan.mark_returned()
    
    def generate_receipt(self, item: LibraryItem, loan: Loan) -> str:
        return f"""
        --- LOAN RECEIPT ---
        {item.receipt_line()}
        Member ID: {loan.member_id}
        Due Date: {loan.due_date.strftime('%Y-%m-%d')}
        Item Type: {item.kind()}
        Media: {item.media_type}
        --------------------
        """



class Catalog:
    def __init__(self):
        self._items: Dict[str, LibraryItem] = {}
    
    def add_item(self, item: LibraryItem) -> None:
        if item.isbn in self._items:
            raise DuplicateBookError(f"ISBN already exists: {item.isbn}")
        if item.copies < 0:
            raise ValueError("copies must be non-negative")
        self._items[item.isbn] = item
    
    def get_item(self, isbn: str) -> Optional[LibraryItem]:
        return self._items.get(isbn)
    
    def list_all_items(self) -> List[LibraryItem]:
        return list(self._items.values())
    
    def filter_by_type(self, kind: str) -> List[LibraryItem]:
        return [item for item in self._items.values() if item.kind() == kind]




## 10) Abstract fee policy

Add an abstract method `daily_late_fee(self) -> float` to `LibraryItem`. Implement fees:
- PrintedBook: 0.25
- EBook: 0.10
- AudioBook: 0.15
Add a concrete `late_fee(self, days_late: int) -> float` in `LibraryItem` that multiplies days by `daily_late_fee()`.

In [ ]:
# Your code here
# Add abstract daily_late_fee and concrete late_fee to LibraryItem and overrides
class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""

    @property
    @abstractmethod
    def media_type(self) -> str:
        """Return the media type as a string."""

    @property
    @abstractmethod
    def borrowable(self) -> bool:
        """Whether the item can be borrowed."""

    @classmethod
    @abstractmethod
    def kind(cls) -> str:
        """Return a short identifier for this kind of library item."""

    @abstractmethod
    def daily_late_fee(self) -> float:
        """Return the daily late fee amount for this item type."""

    def receipt_line(self) -> str:
        return f"{self.describe()} | Loan period: {self.loan_period_days()} days"

    def late_fee(self, days_late: int) -> float:
        if days_late <= 0:
            return 0.0
        return days_late * self.daily_late_fee()

    def due_date_from_today(self) -> datetime:
        return datetime.now() + timedelta(days=self.loan_period_days())

    def can_checkout(self) -> bool:
        return self.copies > 0


class PrintedBook(LibraryItem):
    def loan_period_days(self) -> int:
        return 21
    
    def describe(self) -> str:
        return f"PrintedBook<{self.isbn}>: {self.title} (copies={self.copies})"
    
    @property
    def is_digital(self) -> bool:
        return False
    
    @property
    def media_type(self) -> str:
        return 'print'
    
    @property
    def borrowable(self) -> bool:
        return True
    
    @classmethod
    def kind(cls) -> str:
        return 'book'
    
    def daily_late_fee(self) -> float:
        return 0.25


class EBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 0, file_size_mb: float = 0.0) -> None:
        super().__init__(isbn, title, copies)
        self.file_size_mb = float(file_size_mb)
    
    def loan_period_days(self) -> int:
        return 14
    
    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb:.1f} MB)"
    
    @property
    def is_digital(self) -> bool:
        return True
    
    def can_checkout(self) -> bool:
        return self.copies >= 0
    
    @property
    def media_type(self) -> str:
        return 'ebook'
    
    @property
    def borrowable(self) -> bool:
        return self.copies >= 0
    
    @classmethod
    def kind(cls) -> str:
        return 'ebook'
    
    def download_link(self) -> str:
        return f"https://library.example.com/downloads/ebook/{self.isbn}.epub"
    
    def daily_late_fee(self) -> float:
        return 0.10


class AudioBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 1, duration_min: int = 0) -> None:
        super().__init__(isbn, title, copies)
        self.duration_min = int(duration_min)
    
    def loan_period_days(self) -> int:
        return 14
    
    def describe(self) -> str:
        return f"AudioBook<{self.isbn}>: {self.title} ({self.duration_min} min)"
    
    @property
    def is_digital(self) -> bool:
        return True
    
    @property
    def media_type(self) -> str:
        return 'audiobook'
    
    @property
    def borrowable(self) -> bool:
   
        return True
    
    @classmethod
    def kind(cls) -> str:
        return 'audiobook'
    
    def daily_late_fee(self) -> float:
        """AudioBooks have moderate late fees."""
        return 0.15

## 11) ABC for Member roles

Create an abstract base `MemberRole(ABC)` with `max_concurrent_loans(self) -> int`. Implement `StudentRole` (5) and `StaffRole` (10). Modify `Member` to hold a `role: MemberRole` and delegate `max_concurrent_loans()` to it. Keep the implementation single-inheritance (no mixins).

In [ ]:
# Your code here
# Define roles and update Member

from abc import ABC, abstractmethod

class MemberRole(ABC):
    """Abstract base class defining different membership roles."""
    
    @abstractmethod
    def max_concurrent_loans(self) -> int:
        """Return the maximum number of concurrent loans for this role."""


class StudentRole(MemberRole):
    """Standard student membership with basic borrowing limits."""
    
    def max_concurrent_loans(self) -> int:
        return 5


class StaffRole(MemberRole):
    """Staff membership with elevated borrowing privileges."""
    
    def max_concurrent_loans(self) -> int:
        return 10



@dataclass
class Member:
    member_id: str
    email: str
    role: MemberRole  # Composition: Member HAS-A role
    
    def max_concurrent_loans(self) -> int:
        return self.role.max_concurrent_loans()


print("=== Demonstrating Role-Based Member System ===\n")


student = Member(member_id="S001", email="student@university.edu", role=StudentRole())
staff = Member(member_id="T001", email="staff@university.edu", role=StaffRole())

print("Member Loan Limits:")
print(f"  Student ({student.member_id}): {student.max_concurrent_loans()} concurrent loans")
print(f"  Staff ({staff.member_id}): {staff.max_concurrent_loans()} concurrent loans")
print()


catalog = Catalog()
desk = LoanDesk(catalog)


for i in range(15):
    book = PrintedBook(f'ISBN-{i:03d}', f'Book {i}', copies=1)
    catalog.add_item(book)

print("=== Testing Loan Limits ===\n")


print(f"Student attempting to checkout books (limit: {student.max_concurrent_loans()}):")
student_checkouts = 0
for i in range(7):
    try:
        item = catalog.get_item(f'ISBN-{i:03d}')
        if item:
            loan = desk.checkout(student, item)
            student_checkouts += 1
            print(f"  ✓ Checkout {student_checkouts}: {item.title}")
    except LibraryError as e:
        print(f"  ✗ Checkout {i+1} failed: {e}")
        break
print()


print(f"Staff attempting to checkout books (limit: {staff.max_concurrent_loans()}):")
staff_checkouts = 0
for i in range(7, 15):
    try:
        item = catalog.get_item(f'ISBN-{i:03d}')
        if item:
            loan = desk.checkout(staff, item)
            staff_checkouts += 1
            print(f"  ✓ Checkout {staff_checkouts}: {item.title}")
    except LibraryError as e:
        print(f"  ✗ Checkout {i-6} failed: {e}")
        break
print()


## 12) Prevent partial implementations

Create a subclass `BrokenItem(LibraryItem)` that **forgets** to implement one abstract member. Show that instantiating it raises `TypeError`. Then fix it by implementing the missing member.

In [ ]:
# Your code here
# Show failing instantiation then the fix

print("=== Demonstrating Abstract Method Enforcement ===\n")


class BrokenItem(LibraryItem):
    
    def __init__(self, isbn: str, title: str, copies: int = 1):
        super().__init__(isbn, title, copies)
    
    def loan_period_days(self) -> int:
        return 14
    
    def describe(self) -> str:
        return f"BrokenItem<{self.isbn}>: {self.title}"
    
    @property
    def is_digital(self) -> bool:
        return False
    
    @property
    def media_type(self) -> str:
        return 'broken'
    
    @property
    def borrowable(self) -> bool:
        return True
    
    @classmethod
    def kind(cls) -> str:
        return 'broken'
    



print("Attempting to instantiate BrokenItem (missing daily_late_fee):")
try:
    broken = BrokenItem('999', 'Incomplete Implementation')
    print("  ✗ ERROR: BrokenItem was instantiated (this shouldn't happen!)")
except TypeError as e:
    print(f"  ✓ TypeError caught as expected:")
    print(f"    {e}")
print()


print("Analysis:")
print(f"  BrokenItem has abstract methods: {BrokenItem.__abstractmethods__}")
print(f"  Cannot instantiate until all abstract methods are implemented")
print()


print("=== Fixed Implementation ===\n")

class FixedItem(LibraryItem):

    
    def __init__(self, isbn: str, title: str, copies: int = 1):
        super().__init__(isbn, title, copies)
    
    def loan_period_days(self) -> int:
        return 14
    
    def describe(self) -> str:
        return f"FixedItem<{self.isbn}>: {self.title}"
    
    @property
    def is_digital(self) -> bool:
        return False
    
    @property
    def media_type(self) -> str:
        return 'fixed'
    
    @property
    def borrowable(self) -> bool:
        return True
    
    @classmethod
    def kind(cls) -> str:
        return 'fixed'
    

    def daily_late_fee(self) -> float:
        return 0.20



print("Attempting to instantiate FixedItem (all methods implemented):")
try:
    fixed = FixedItem('888', 'Complete Implementation', copies=5)
    print(f"  ✓ Success! Created: {fixed.describe()}")
    print(f"    Loan period: {fixed.loan_period_days()} days")
    print(f"    Daily late fee: ${fixed.daily_late_fee():.2f}")
    print(f"    Late fee for 10 days: ${fixed.late_fee(10):.2f}")
    print(f"    Kind: {fixed.kind()}")
    print(f"    Borrowable: {fixed.borrowable}")
except TypeError as e:
    print(f"  ✗ Unexpected error: {e}")
print()


print("Verification:")
print(f"  FixedItem has abstract methods: {FixedItem.__abstractmethods__}")
print(f"  FixedItem can be instantiated: {len(FixedItem.__abstractmethods__) == 0}")
print()


## 13) Abstract validation hook

Add an abstract hook `validate_on_add(self) -> None` to `LibraryItem` and override it in each subclass to enforce a simple constraint (e.g., `copies >= 0`). Modify `Catalog.add_item` to call `item.validate_on_add()` before insertion.

In [ ]:
# Your code here
# Add validate_on_add to classes and call from Catalog.add_item

class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""

    @property
    @abstractmethod
    def media_type(self) -> str:
        """Return the media type as a string."""

    @property
    @abstractmethod
    def borrowable(self) -> bool:
        """Whether the item can be borrowed."""

    @classmethod
    @abstractmethod
    def kind(cls) -> str:
        """Return a short identifier for this kind of library item."""

    @abstractmethod
    def daily_late_fee(self) -> float:
        """Return the daily late fee amount for this item type."""

    @abstractmethod
    def validate_on_add(self) -> None:
        """Validate item before adding to catalog. Raises ValueError if invalid."""


    def receipt_line(self) -> str:
        """Generate a receipt line using describe() and loan_period_days()."""
        return f"{self.describe()} | Loan period: {self.loan_period_days()} days"


    def late_fee(self, days_late: int) -> float:
        if days_late <= 0:
            return 0.0
        return days_late * self.daily_late_fee()


    def due_date_from_today(self) -> datetime:
        return datetime.now() + timedelta(days=self.loan_period_days())


    def can_checkout(self) -> bool:
        return self.copies > 0


class PrintedBook(LibraryItem):
    def loan_period_days(self) -> int:
        return 21
    
    def describe(self) -> str:
        return f"PrintedBook<{self.isbn}>: {self.title} (copies={self.copies})"
    
    @property
    def is_digital(self) -> bool:
        return False
    
    @property
    def media_type(self) -> str:
        return 'print'
    
    @property
    def borrowable(self) -> bool:
        return True
    
    @classmethod
    def kind(cls) -> str:
        return 'book'
    
    def daily_late_fee(self) -> float:
        return 0.25
    
    def validate_on_add(self) -> None:
        if self.copies < 0:
            raise ValueError(f"PrintedBook copies must be >= 0, got {self.copies}")
        if self.copies == 0:
            raise ValueError(f"PrintedBook must have at least 1 physical copy")
        if not self.isbn or len(self.isbn) < 5:
            raise ValueError(f"PrintedBook must have valid ISBN (min 5 chars)")
        if not self.title.strip():
            raise ValueError(f"PrintedBook must have a non-empty title")


class EBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 0, file_size_mb: float = 0.0) -> None:
        super().__init__(isbn, title, copies)
        self.file_size_mb = float(file_size_mb)
    
    def loan_period_days(self) -> int:
        return 14
    
    def describe(self) -> str:
        return f"EBook<{self.isbn}>: {self.title} ({self.file_size_mb:.1f} MB)"
    
    @property
    def is_digital(self) -> bool:
        return True
    
    def can_checkout(self) -> bool:
        return self.copies >= 0
    
    @property
    def media_type(self) -> str:
        return 'ebook'
    
    @property
    def borrowable(self) -> bool:
        return self.copies >= 0
    
    @classmethod
    def kind(cls) -> str:
        return 'ebook'
    
    def download_link(self) -> str:
        return f"https://library.example.com/downloads/ebook/{self.isbn}.epub"
    
    def daily_late_fee(self) -> float:
        return 0.10
    
    def validate_on_add(self) -> None:
        if self.copies < 0:
            raise ValueError(f"EBook licenses (copies) must be >= 0, got {self.copies}")
        if self.file_size_mb <= 0:
            raise ValueError(f"EBook file_size_mb must be > 0, got {self.file_size_mb}")
        if self.file_size_mb > 1000:
            raise ValueError(f"EBook file_size_mb exceeds maximum (1000 MB), got {self.file_size_mb}")
        if not self.title.strip():
            raise ValueError(f"EBook must have a non-empty title")


class AudioBook(LibraryItem):
    def __init__(self, isbn: str, title: str, copies: int = 1, duration_min: int = 0) -> None:
        super().__init__(isbn, title, copies)
        self.duration_min = int(duration_min)
    
    def loan_period_days(self) -> int:
        return 14
    
    def describe(self) -> str:
        return f"AudioBook<{self.isbn}>: {self.title} ({self.duration_min} min)"
    
    @property
    def is_digital(self) -> bool:
        return True
    
    @property
    def media_type(self) -> str:
        return 'audiobook'
    
    @property
    def borrowable(self) -> bool:
        return True
    
    @classmethod
    def kind(cls) -> str:
        return 'audiobook'
    
    def daily_late_fee(self) -> float:
        return 0.15
    
    def validate_on_add(self) -> None:
        if self.copies < 0:
            raise ValueError(f"AudioBook copies must be >= 0, got {self.copies}")
        if self.duration_min <= 0:
            raise ValueError(f"AudioBook duration must be > 0 minutes, got {self.duration_min}")
        if self.duration_min > 10000:
            raise ValueError(f"AudioBook duration exceeds maximum (10000 min), got {self.duration_min}")
        if not self.title.strip():
            raise ValueError(f"AudioBook must have a non-empty title")


class Catalog:
    def __init__(self):
        self._items: Dict[str, LibraryItem] = {}
    
    def add_item(self, item: LibraryItem) -> None:
        if item.isbn in self._items:
            raise DuplicateBookError(f"ISBN already exists: {item.isbn}")
        
        item.validate_on_add()
        
        self._items[item.isbn] = item
    
    def get_item(self, isbn: str) -> Optional[LibraryItem]:
        return self._items.get(isbn)
    
    def list_all_items(self) -> List[LibraryItem]:
        return list(self._items.values())


## 14) Minimal adapter via ABC registration

Suppose you receive third-party objects with attributes `code`, `name`, `stock` that you want to treat as `LibraryItem`. Write a light Adapter class that **implements** the `LibraryItem` API and delegates to the 3rd-party object, then register it (or the 3rd-party class) appropriately. Show it working with `LoanDesk.checkout`.

In [ ]:
# Your code here
# Implement Adapter that conforms to LibraryItem contract

class ThirdPartyItem:
    def __init__(self, code: str, name: str, stock: int, item_type: str = 'generic'):
        self.code = code  
        self.name = name  
        self.stock = stock  
        self.item_type = item_type 
    
    def __repr__(self):
        return f"ThirdPartyItem(code={self.code}, name={self.name}, stock={self.stock})"



class ThirdPartyItemAdapter(LibraryItem):
    
    def __init__(self, third_party_item: ThirdPartyItem):
        self._adapted = third_party_item
        
  
        super().__init__(
            isbn=third_party_item.code,
            title=third_party_item.name,
            copies=third_party_item.stock
        )
    
    def loan_period_days(self) -> int:
        type_mapping = {
            'book': 21,
            'digital': 14,
            'audio': 14,
            'generic': 14
        }
        return type_mapping.get(self._adapted.item_type, 14)
    
    def describe(self) -> str:
        return f"ThirdPartyAdapter<{self.isbn}>: {self.title} (stock={self.copies}, type={self._adapted.item_type})"
    
    @property
    def is_digital(self) -> bool:
        return self._adapted.item_type in ['digital', 'audio']
    
    @property
    def media_type(self) -> str:
        return f'adapted_{self._adapted.item_type}'
    
    @property
    def borrowable(self) -> bool:
        return self.copies > 0
    
    @classmethod
    def kind(cls) -> str:
        return 'adapted'
    
    def daily_late_fee(self) -> float:
        fee_mapping = {
            'book': 0.25,
            'digital': 0.10,
            'audio': 0.15,
            'generic': 0.20
        }
        return fee_mapping.get(self._adapted.item_type, 0.20)
    
    def validate_on_add(self) -> None:
        if self.copies < 0:
            raise ValueError(f"Adapted item stock must be >= 0, got {self.copies}")
        if not self.title.strip():
            raise ValueError(f"Adapted item must have a non-empty name")
    
    @property
    def copies(self) -> int:
        return self._adapted.stock
    
    @copies.setter
    def copies(self, value: int) -> None:
        self._adapted.stock = value


## 15) Unit test: abstract contract

Using `unittest`, write tests that assert:
- `LibraryItem` instantiation fails
- All concrete classes implement `loan_period_days` and `describe`
- `late_fee` uses the subclass-specific `daily_late_fee` values

In [ ]:
# Your code here
import unittest

class TestWeek9ABCs(unittest.TestCase):
    
    def test_abstract_instantiation(self):
        with self.assertRaises(TypeError) as context:
            item = LibraryItem('123', 'Should Fail')
        error_msg = str(context.exception)
        self.assertIn("abstract", error_msg.lower())
        self.assertIn("LibraryItem", error_msg)
    
    def test_concretes_implement_contract(self):
        book = PrintedBook('111', 'Test Book', copies=3)
        ebook = EBook('222', 'Test EBook', copies=5, file_size_mb=2.5)
        audio = AudioBook('333', 'Test Audio', copies=2, duration_min=60)
        
        concrete_items = [book, ebook, audio]
        
        for item in concrete_items:
            self.assertTrue(hasattr(item, 'loan_period_days'))
            self.assertTrue(callable(item.loan_period_days))
            loan_period = item.loan_period_days()
            self.assertIsInstance(loan_period, int)
            self.assertGreater(loan_period, 0)
            
            self.assertTrue(hasattr(item, 'describe'))
            self.assertTrue(callable(item.describe))
            description = item.describe()
            self.assertIsInstance(description, str)
            self.assertGreater(len(description), 0)
            
            self.assertTrue(hasattr(item, 'is_digital'))
            self.assertIsInstance(item.is_digital, bool)
            
            self.assertTrue(hasattr(item, 'media_type'))
            self.assertIsInstance(item.media_type, str)
            
            self.assertTrue(hasattr(item, 'borrowable'))
            self.assertIsInstance(item.borrowable, bool)
            
            self.assertTrue(hasattr(item, 'kind'))
            self.assertTrue(callable(item.kind))
            self.assertIsInstance(item.kind(), str)
            
            self.assertTrue(hasattr(item, 'daily_late_fee'))
            self.assertTrue(callable(item.daily_late_fee))
            self.assertIsInstance(item.daily_late_fee(), float)
            
            self.assertTrue(hasattr(item, 'validate_on_add'))
            self.assertTrue(callable(item.validate_on_add))
    
    def test_late_fee_polymorphism(self):
        book = PrintedBook('111', 'Test Book', copies=3)
        ebook = EBook('222', 'Test EBook', copies=5, file_size_mb=2.5)
        audio = AudioBook('333', 'Test Audio', copies=2, duration_min=60)
        
        self.assertEqual(book.daily_late_fee(), 0.25)
        self.assertEqual(ebook.daily_late_fee(), 0.10)
        self.assertEqual(audio.daily_late_fee(), 0.15)
        
        days_late = 10

        expected_book_fee = days_late * 0.25
        actual_book_fee = book.late_fee(days_late)
        self.assertAlmostEqual(actual_book_fee, expected_book_fee, places=2)
        
        expected_ebook_fee = days_late * 0.10
        actual_ebook_fee = ebook.late_fee(days_late)
        self.assertAlmostEqual(actual_ebook_fee, expected_ebook_fee, places=2)
        
        expected_audio_fee = days_late * 0.15
        actual_audio_fee = audio.late_fee(days_late)
        self.assertAlmostEqual(actual_audio_fee, expected_audio_fee, places=2)
        
        self.assertEqual(book.late_fee(0), 0.0)
        self.assertEqual(ebook.late_fee(0), 0.0)
        self.assertEqual(audio.late_fee(0), 0.0)
        

        self.assertEqual(book.late_fee(-5), 0.0)
        self.assertEqual(ebook.late_fee(-5), 0.0)
        self.assertEqual(audio.late_fee(-5), 0.0)
        
        days = 7
        book_fee = book.late_fee(days)
        ebook_fee = ebook.late_fee(days)
        audio_fee = audio.late_fee(days)
        
        self.assertNotEqual(book_fee, ebook_fee)
        self.assertNotEqual(book_fee, audio_fee)
        self.assertNotEqual(ebook_fee, audio_fee)
        
        self.assertGreater(book_fee, audio_fee)
        self.assertGreater(audio_fee, ebook_fee)
    
    def test_template_method_pattern(self):
        items = [
            PrintedBook('111', 'Book', copies=1),
            EBook('222', 'EBook', copies=1, file_size_mb=1.0),
            AudioBook('333', 'Audio', copies=1, duration_min=60)
        ]
        
        for item in items:
            self.assertTrue(hasattr(item, 'late_fee'))
            self.assertEqual(item.late_fee.__name__, 'late_fee')
    
    def test_abstract_properties(self):
        book = PrintedBook('111', 'Test', copies=1)
        
        self.assertFalse(book.is_digital)
        
  
        self.assertEqual(book.media_type, 'print')
     
        self.assertTrue(book.borrowable)
    
    def test_abstract_classmethod(self):
        self.assertEqual(PrintedBook.kind(), 'book')
        self.assertEqual(EBook.kind(), 'ebook')
        self.assertEqual(AudioBook.kind(), 'audiobook')
        
        book = PrintedBook('111', 'Test', copies=1)
        ebook = EBook('222', 'Test', copies=1, file_size_mb=1.0)
        audio = AudioBook('333', 'Test', copies=1, duration_min=60)
        
        self.assertEqual(book.kind(), 'book')
        self.assertEqual(ebook.kind(), 'ebook')
        self.assertEqual(audio.kind(), 'audiobook')
    
    def test_validate_on_add_hook(self):
        # Valid items should not raise
        book = PrintedBook('111', 'Valid Book', copies=1)
        try:
            book.validate_on_add()
        except ValueError:
            self.fail("validate_on_add raised ValueError for valid item")
        
        invalid_book = PrintedBook('111', 'Invalid', copies=-1)
        with self.assertRaises(ValueError):
            invalid_book.validate_on_add()
        
        invalid_ebook = EBook('222', 'Invalid', copies=1, file_size_mb=-5.0)
        with self.assertRaises(ValueError):
            invalid_ebook.validate_on_add()
        
        invalid_audio = AudioBook('333', 'Invalid', copies=1, duration_min=0)
        with self.assertRaises(ValueError):
            invalid_audio.validate_on_add()
    
    def test_isinstance_with_abc(self):
        book = PrintedBook('111', 'Test', copies=1)
        ebook = EBook('222', 'Test', copies=1, file_size_mb=1.0)
        audio = AudioBook('333', 'Test', copies=1, duration_min=60)
        
        self.assertIsInstance(book, LibraryItem)
        self.assertIsInstance(ebook, LibraryItem)
        self.assertIsInstance(audio, LibraryItem)
        
  
        self.assertIsInstance(book, PrintedBook)
        self.assertIsInstance(ebook, EBook)
        self.assertIsInstance(audio, AudioBook)
        
   
        self.assertNotIsInstance(book, EBook)
        self.assertNotIsInstance(ebook, AudioBook)
        self.assertNotIsInstance(audio, PrintedBook)




## 16) Swap implementation behind the ABC

Write a function `checkout_any(desk: LoanDesk, member: Member, item: LibraryItem)` that works for **any** `LibraryItem` or registered virtual subclass. Demonstrate with a `PDFPamphlet` instance (from Ex. 5) and a normal `PrintedBook`.

In [ ]:
# Your code here
def checkout_any(desk: LoanDesk, member: Member, item: LibraryItem) -> Loan:
    if not isinstance(item, LibraryItem):
        raise TypeError(f"Expected LibraryItem, got {type(item).__name__}")
    
    print(f"  Processing checkout for {member.member_id}:")
    print(f"    Item: {item.describe()}")
    print(f"    Type: {item.__class__.__name__} (kind: {item.kind()})")
    print(f"    Loan period: {item.loan_period_days()} days")
    print(f"    Daily late fee: ${item.daily_late_fee():.2f}")
    
    loan = desk.checkout(member, item)
    
    print(f"     Checkout successful!")
    print(f"    Due date: {loan.due_date.strftime('%Y-%m-%d')}")
    
    return loan


print("=== Demonstrating checkout_any with Polymorphism ===\n")

catalog = Catalog()
desk = LoanDesk(catalog)
member = Member(member_id="M001", email="user@example.com", role=StudentRole())

print("1. Creating PrintedBook (direct subclass):")
printed_book = PrintedBook('978-0-123456-78-9', 'Python Programming', copies=5)
catalog.add_item(printed_book)
print(f"   {printed_book.describe()}")
print(f"   isinstance(printed_book, LibraryItem): {isinstance(printed_book, LibraryItem)}")
print()

print("2. Creating PDFPamphlet (virtual subclass):")
pdf_pamphlet = PDFPamphlet('978-9-999999-99-9', 'Quick Reference Guide', copies=10, pages=8)
LibraryItem.register(PDFPamphlet)
catalog.add_item(pdf_pamphlet)
print(f"   {pdf_pamphlet.describe()}")
print(f"   isinstance(pdf_pamphlet, LibraryItem): {isinstance(pdf_pamphlet, LibraryItem)}")
print(f"   PDFPamphlet in LibraryItem.__subclasses__(): {PDFPamphlet in LibraryItem.__subclasses__()}")
print(f"   issubclass(PDFPamphlet, LibraryItem): {issubclass(PDFPamphlet, LibraryItem)}")
print()

print("3. Creating additional items:")
ebook = EBook('978-0-987654-32-1', 'Django Web Development', copies=3, file_size_mb=4.5)
audio = AudioBook('978-1-555555-55-5', 'Python Podcast', copies=2, duration_min=180)
catalog.add_item(ebook)
catalog.add_item(audio)
print(f"   {ebook.describe()}")
print(f"   {audio.describe()}")
print()
print("=== Testing checkout_any Function ===\n")

print("Test 1: Checkout PrintedBook")
try:
    loan1 = checkout_any(desk, member, printed_book)
except LibraryError as e:
    print(f"   Checkout failed: {e}")
print()


print("Test 2: Checkout PDFPamphlet (virtual subclass)")
try:
    loan2 = checkout_any(desk, member, pdf_pamphlet)
except LibraryError as e:
    print(f"   Checkout failed: {e}")
print()

print("Test 3: Checkout EBook")
try:
    loan3 = checkout_any(desk, member, ebook)
except LibraryError as e:
    print(f"   Checkout failed: {e}")
print()


print("Test 4: Checkout AudioBook")
try:
    loan4 = checkout_any(desk, member, audio)
except LibraryError as e:
    print(f"   Checkout failed: {e}")
print()


print("=== Active Loans Summary ===")
active_loans = desk.active_loans_for(member)
print(f"Member {member.member_id} has {len(active_loans)} active loan(s):\n")
for i, loan in enumerate(active_loans, 1):
    item = catalog.get_item(loan.isbn)
    if item:
        print(f"{i}. {item.describe()}")
        print(f"   Due: {loan.due_date.strftime('%Y-%m-%d')}")
        print(f"   Late fee if 5 days late: ${item.late_fee(5):.2f}")
        print()

print("=== Type Safety Test ===\n")

class NotALibraryItem:
    def __init__(self, name):
        self.name = name

print("Test: Attempting to checkout non-LibraryItem object")
fake_item = NotALibraryItem("Fake Book")
try:
    checkout_any(desk, member, fake_item)
    print("   ERROR: checkout_any accepted invalid type!")
except TypeError as e:
    print(f"   Type check caught invalid object: {e}")
print()


print("=== Polymorphic Collection Processing ===\n")


member2 = Member(member_id="M002", email="staff@example.com", role=StaffRole())


items_to_checkout = [
    PrintedBook('BATCH-001', 'Book 1', copies=1),
    PDFPamphlet('BATCH-002', 'Pamphlet 1', copies=1, pages=5),
    EBook('BATCH-003', 'EBook 1', copies=1, file_size_mb=2.0),
    AudioBook('BATCH-004', 'Audio 1', copies=1, duration_min=90)
]


for item in items_to_checkout:
    catalog.add_item(item)

print(f"Batch checkout for {member2.member_id} ({member2.role.__class__.__name__}):")
successful_checkouts = 0
for item in items_to_checkout:
    try:
        loan = checkout_any(desk, member2, item)
        successful_checkouts += 1
    except LibraryError as e:
        print(f"  Failed: {e}")
    print()

print(f"Batch checkout complete: {successful_checkouts}/{len(items_to_checkout)} successful\n")


## 17) Abstract property + computed template

Add a template method `full_label()` to `LibraryItem` that returns `f"[{self.media_type}] {self.title} — {self.isbn}"`. Confirm each subclass inherits it and shows the right `media_type` value.

In [ ]:
# Your code here
# Implement full_label in LibraryItem and demonstrate across subclasses

class LibraryItem(ABC):
    def __init__(self, isbn: str, title: str, copies: int = 1) -> None:
        self.isbn = isbn
        self.title = title
        self.copies = copies

    @abstractmethod
    def loan_period_days(self) -> int:
        """Each concrete item defines its loan period."""

    @abstractmethod
    def describe(self) -> str:
        """Human-readable description of the item."""

    @property
    @abstractmethod
    def is_digital(self) -> bool:
        """Whether the item is digital."""

    @property
    @abstractmethod
    def media_type(self) -> str:
        """Return the media type as a string."""

    @property
    @abstractmethod
    def borrowable(self) -> bool:
        """Whether the item can be borrowed."""

    @classmethod
    @abstractmethod
    def kind(cls) -> str:
        """Return a short identifier for this kind of library item."""

    @abstractmethod
    def daily_late_fee(self) -> float:
        """Return the daily late fee amount for this item type."""

    @abstractmethod
    def validate_on_add(self) -> None:
        """Validate item before adding to catalog. Raises ValueError if invalid."""

  
    def full_label(self) -> str:
        return f"[{self.media_type}] {self.title} — {self.isbn}"

    def receipt_line(self) -> str:
        return f"{self.describe()} | Loan period: {self.loan_period_days()} days"

    def late_fee(self, days_late: int) -> float:
        if days_late <= 0:
            return 0.0
        return days_late * self.daily_late_fee()

    def due_date_from_today(self) -> datetime:
        return datetime.now() + timedelta(days=self.loan_period_days())


    def can_checkout(self) -> bool:
        return self.copies > 0


## 18) issubclass / isinstance with ABCs

Show examples of `issubclass(PrintedBook, LibraryItem)` and `isinstance(EBook(...), LibraryItem)`. After registering a virtual subclass, show `issubclass(PDFPamphlet, LibraryItem)` is `True` as well.

In [ ]:
# Your code here
# Demonstrate issubclass/isinstance facts

print("=== Demonstrating issubclass() and isinstance() ===\n")

print("PART 1: Real Subclasses (Direct Inheritance)")
print("=" * 70)

print("\n1. Using issubclass() with real subclasses:")
print(f"   issubclass(PrintedBook, LibraryItem) = {issubclass(PrintedBook, LibraryItem)}")
print(f"   issubclass(EBook, LibraryItem) = {issubclass(EBook, LibraryItem)}")
print(f"   issubclass(AudioBook, LibraryItem) = {issubclass(AudioBook, LibraryItem)}")


print("\n2. Checking class hierarchy (MRO - Method Resolution Order):")
print(f"   PrintedBook.__bases__ = {PrintedBook.__bases__}")
print(f"   PrintedBook.__mro__ = {[c.__name__ for c in PrintedBook.__mro__]}")


print("\n3. Using isinstance() with instances:")
ebook_instance = EBook('978-0-123456-78-9', 'Django Guide', copies=5, file_size_mb=3.2)
print(f"   Created: {ebook_instance.describe()}")
print(f"   isinstance(ebook_instance, EBook) = {isinstance(ebook_instance, EBook)}")
print(f"   isinstance(ebook_instance, LibraryItem) = {isinstance(ebook_instance, LibraryItem)}")
print(f"   isinstance(ebook_instance, ABC) = {isinstance(ebook_instance, ABC)}")

book_instance = PrintedBook('978-1-111111-11-1', 'Python Basics', copies=3)
audio_instance = AudioBook('978-2-222222-22-2', 'Learn Audio', copies=2, duration_min=120)

print(f"\n   isinstance(book_instance, LibraryItem) = {isinstance(book_instance, LibraryItem)}")
print(f"   isinstance(audio_instance, LibraryItem) = {isinstance(audio_instance, LibraryItem)}")


print("\n4. Cross-type isinstance checks (should be False):")
print(f"   isinstance(ebook_instance, PrintedBook) = {isinstance(ebook_instance, PrintedBook)}")
print(f"   isinstance(book_instance, EBook) = {isinstance(book_instance, EBook)}")
print(f"   isinstance(audio_instance, PrintedBook) = {isinstance(audio_instance, PrintedBook)}")

print("\n" + "=" * 70)

print("\n\nPART 2: Virtual Subclasses - BEFORE Registration")
print("=" * 70)

print("\n1. PDFPamphlet class definition (no inheritance from LibraryItem):")
print(f"   PDFPamphlet.__bases__ = {PDFPamphlet.__bases__}")
print(f"   PDFPamphlet.__mro__ = {[c.__name__ for c in PDFPamphlet.__mro__]}")

print("\n2. Checking relationships BEFORE registration:")
print(f"   issubclass(PDFPamphlet, LibraryItem) = {issubclass(PDFPamphlet, LibraryItem)}")
print(f"   LibraryItem in PDFPamphlet.__mro__ = {LibraryItem in PDFPamphlet.__mro__}")

pdf_instance = PDFPamphlet('978-9-999999-99-9', 'Quick Reference', copies=10, pages=8)
print(f"\n3. Created PDFPamphlet instance: {pdf_instance.describe()}")
print(f"   isinstance(pdf_instance, PDFPamphlet) = {isinstance(pdf_instance, PDFPamphlet)}")
print(f"   isinstance(pdf_instance, LibraryItem) = {isinstance(pdf_instance, LibraryItem)}")

print("\n" + "=" * 70)


print("\n\nPART 3: Virtual Subclasses - AFTER Registration")
print("=" * 70)

print("\n1. Registering PDFPamphlet as a virtual subclass:")
print("   >>> LibraryItem.register(PDFPamphlet)")
LibraryItem.register(PDFPamphlet)
print("   ✓ Registration complete!")

print("\n2. Checking relationships AFTER registration:")
print(f"   issubclass(PDFPamphlet, LibraryItem) = {issubclass(PDFPamphlet, LibraryItem)}")
print(f"   isinstance(pdf_instance, LibraryItem) = {isinstance(pdf_instance, LibraryItem)}")

print("\n3. BUT the actual inheritance hasn't changed:")
print(f"   PDFPamphlet.__bases__ = {PDFPamphlet.__bases__}")
print(f"   PDFPamphlet.__mro__ = {[c.__name__ for c in PDFPamphlet.__mro__]}")
print(f"   LibraryItem in PDFPamphlet.__mro__ = {LibraryItem in PDFPamphlet.__mro__}")

print("\n4. Virtual subclass is NOT in __subclasses__():")
real_subclasses = LibraryItem.__subclasses__()
print(f"   LibraryItem.__subclasses__() = {[c.__name__ for c in real_subclasses]}")
print(f"   PDFPamphlet in LibraryItem.__subclasses__() = {PDFPamphlet in real_subclasses}")

print("\n" + "=" * 70)


print("\n\nPART 4: Comprehensive Comparison Table")
print("=" * 70)

classes_to_test = [
    ('PrintedBook', PrintedBook),
    ('EBook', EBook),
    ('AudioBook', AudioBook),
    ('PDFPamphlet', PDFPamphlet)
]

print(f"\n{'Class':<20} {'issubclass':<12} {'In __mro__':<12} {'In __subclasses__':<18} {'Inheritance Type'}")
print("-" * 80)

for name, cls in classes_to_test:
    is_subclass = issubclass(cls, LibraryItem)
    in_mro = LibraryItem in cls.__mro__
    in_subclasses = cls in LibraryItem.__subclasses__()
    inheritance = "Direct" if in_mro else "Virtual"
    
    print(f"{name:<20} {str(is_subclass):<12} {str(in_mro):<12} {str(in_subclasses):<18} {inheritance}")

print("-" * 80)

print("\n\nPART 5: Practical Demonstration - Type Checking Function")
print("=" * 70)

def process_library_item(item: LibraryItem) -> None:
    """Function that accepts any LibraryItem (including virtual subclasses)."""
    if not isinstance(item, LibraryItem):
        raise TypeError(f"Expected LibraryItem, got {type(item).__name__}")
    
    print(f"   ✓ Processing: {item.full_label()}")
    print(f"     Type: {item.__class__.__name__} (kind: {item.kind()})")
    print(f"     Inheritance: {'Direct' if LibraryItem in item.__class__.__mro__ else 'Virtual'}")

print("\n1. Processing various item types:\n")

test_items = [
    PrintedBook('TEST-001', 'Test Book', copies=1),
    EBook('TEST-002', 'Test EBook', copies=1, file_size_mb=1.0),
    AudioBook('TEST-003', 'Test Audio', copies=1, duration_min=60),
    PDFPamphlet('TEST-004', 'Test Pamphlet', copies=1, pages=5)
]

for item in test_items:
    process_library_item(item)
    print()


print("=" * 70)
print("\nPART 6: Understanding How Virtual Subclasses Work")
print("=" * 70)

print("""
Virtual subclass registration uses the ABCMeta metaclass to maintain
a registry of "fake" subclasses. When you call:

    LibraryItem.register(PDFPamphlet)

The ABCMeta metaclass:
1. Adds PDFPamphlet to an internal _abc_registry
2. Makes isinstance() and issubclass() return True
3. Does NOT modify the actual class hierarchy (__mro__)
4. Does NOT add PDFPamphlet to __subclasses__()

This allows duck typing with explicit registration!
""")

print("\n" + "=" * 70)


print("\nPART 7: Advanced Type Checking")
print("=" * 70)

print("\n1. Type hierarchy verification:")

def check_type_relationships(cls, name):
    print(f"\n{name}:")
    print(f"  is subclass of LibraryItem: {issubclass(cls, LibraryItem)}")
    print(f"  is subclass of ABC: {issubclass(cls, ABC)}")
    print(f"  direct bases: {[c.__name__ for c in cls.__bases__]}")
    
    
    is_true_subclass = LibraryItem in cls.__mro__
    print(f"  true subclass (in MRO): {is_true_subclass}")
    print(f"  type: {'Direct inheritance' if is_true_subclass else 'Virtual subclass'}")

check_type_relationships(PrintedBook, "PrintedBook")
check_type_relationships(PDFPamphlet, "PDFPamphlet (registered virtual)")

print("\n2. Creating instances and testing isinstance:")
print(f"\n   book = PrintedBook('B1', 'Title', copies=1)")
book = PrintedBook('B1', 'Title', copies=1)
print(f"   isinstance(book, LibraryItem): {isinstance(book, LibraryItem)}")
print(f"   type(book).__mro__: {[c.__name__ for c in type(book).__mro__]}")

print(f"\n   pdf = PDFPamphlet('P1', 'Title', copies=1, pages=5)")
pdf = PDFPamphlet('P1', 'Title', copies=1, pages=5)
print(f"   isinstance(pdf, LibraryItem): {isinstance(pdf, LibraryItem)}")
print(f"   type(pdf).__mro__: {[c.__name__ for c in type(pdf).__mro__]}")

print("\n" + "=" * 70)
print("\n=== SUMMARY ===")
print("""
Key Takeaways:

1. REAL SUBCLASSES (Direct Inheritance):
   ✓ issubclass(PrintedBook, LibraryItem) → True
   ✓ isinstance(ebook_instance, LibraryItem) → True
   ✓ LibraryItem appears in __mro__
   ✓ Class appears in LibraryItem.__subclasses__()

2. VIRTUAL SUBCLASSES (After Registration):
   ✓ issubclass(PDFPamphlet, LibraryItem) → True (after register)
   ✓ isinstance(pdf_instance, LibraryItem) → True (after register)
   ✗ LibraryItem does NOT appear in __mro__
   ✗ Class does NOT appear in LibraryItem.__subclasses__()

3. PRACTICAL IMPACT:
   • Both work identically with isinstance() and issubclass()
   • Both can be used with type hints: item: LibraryItem
   • Virtual subclasses don't inherit implementation
   • Virtual subclasses must implement interface manually
   • Useful for third-party classes you can't modify

4. USE CASES:
   • Direct inheritance: When you control the class
   • Virtual subclass: Third-party classes, legacy code, duck typing
""")
print("=" * 70)

## 19) Inventory report via abstraction

Write `summarize_items(items: list[LibraryItem]) -> list[str]` that uses only abstract methods/properties (`describe`, `media_type`, etc.). Demonstrate polymorphism by passing a mixed list of items.

In [ ]:
# Your code here
def summarize_items(items: list[LibraryItem]) -> list[str]:
    summaries = []
    for item in items:
        summaries.append(f"{item.media_type}: {item.describe()}")
    return summaries

## 20) End-to-end scenario under ABC contract

Create a demo that:
- Builds a `Catalog` and `LoanDesk`
- Adds one of each concrete item and one registered virtual subclass instance
- Checks each out to a `Member`
- Prints a small receipt using `receipt_line()` and the computed due dates
Use only `LibraryItem`-level APIs at call sites (no `isinstance` branches).

In [ ]:
from abc import ABC, abstractmethod
from datetime import datetime, timedelta



class LibraryItem(ABC):
    @property
    @abstractmethod
    def media_type(self) -> str:
        ...

    @abstractmethod
    def describe(self) -> str:
        ...

    @property
    @abstractmethod
    def checkout_duration_days(self) -> int:
        ...

    def receipt_line(self, due: datetime) -> str:
        return f"{self.media_type}: {self.describe()} — Due {due.date()}"



class Book(LibraryItem):
    def __init__(self, title, author):
        self.title = title
        self.author = author

    @property
    def media_type(self) -> str:
        return "Book"

    def describe(self) -> str:
        return f"{self.title} by {self.author}"

    @property
    def checkout_duration_days(self) -> int:
        return 21


class Movie(LibraryItem):
    def __init__(self, title, director):
        self.title = title
        self.director = director

    @property
    def media_type(self) -> str:
        return "Movie"

    def describe(self) -> str:
        return f"{self.title}, dir. {self.director}"

    @property
    def checkout_duration_days(self) -> int:
        return 7



class Game:
    def __init__(self, title, console):
        self.title = title
        self.console = console

    @property
    def media_type(self) -> str:
        return "Game"

    def describe(self) -> str:
        return f"{self.title} on {self.console}"

    @property
    def checkout_duration_days(self) -> int:
        return 14

    def receipt_line(self, due: datetime) -> str:
        return f"Game: {self.title} ({self.console}) — Due {due.date()}"


LibraryItem.register(Game)




class Member:
    def __init__(self, name):
        self.name = name


class Catalog:
    def __init__(self):
        self.items = []

    def add(self, item: LibraryItem) -> None:
        self.items.append(item)


class Loan:
    def __init__(self, item: LibraryItem, member: Member, out_date: datetime):
        self.item = item
        self.member = member
        self.out_date = out_date
        self.due = out_date + timedelta(days=item.checkout_duration_days)


class LoanDesk:
    def checkout(self, item: LibraryItem, member: Member) -> Loan:
        return Loan(item, member, datetime.now())




def demo():
    catalog = Catalog()
    desk = LoanDesk()
    member = Member("Alice")

    book = Book("Dune", "Frank Herbert")
    movie = Movie("Inception", "Christopher Nolan")

    game = Game("Zelda: Breath of the Wild", "Switch")

    
    catalog.add(book)
    catalog.add(movie)
    catalog.add(game)

    print("=== RECEIPT ===")
    for item in catalog.items:
        loan = desk.checkout(item, member)
        print(item.receipt_line(loan.due))



demo()


## Python skills you'll need (Weeks 1–9)

- **Core syntax & data types:** variables, strings, numbers, booleans
- **Collections:** lists, dicts (basic use), simple comprehensions
- **Control flow:** `if/elif/else`, `for`, `while`
- **Functions & modules:** defining functions, parameters, returns, imports
- **File I/O & JSON (basic):** open/read/write, simple JSON usage
- **Classes & objects (Weeks 4–8):** classes, `__init__`, instance methods, overriding, `super()`
- **Encapsulation basics:** simple validation; naming conventions for "private" attributes
- **Error handling & testing (Week 7):** `try/except`, custom exceptions, basic `unittest`
- **Week 8 OOP:** single inheritance & polymorphism (no ABCs)
- **Week 9 focus:** **Abstract Base Classes (ABC)** with `abc.ABC` and `@abstractmethod`, abstract properties, class/instance abstract methods, virtual subclass **registration**, and light `typing.Protocol` usage (non-generic)
- **Standard library familiarity:** `abc`, `datetime`, built-in exceptions
